In [ ]:
from google.colab import files
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
uploaded = files.upload() #this imports the excel file
cols = ["Institution", "Degree Level", "Major", "Major code", "Cohort Year", 
"Graduates (with full-time wages)", 
"25th percentile earnings: full-time workers", 
"Median earnings: full-time workers", 
"75th percentile earnings: full-time workers", 
"Percent of graduates continuing education", "Percent working full-time", 
"Median earnings: Bachelor's degree earners with no additional credential", 
"Percent of bachelor's degree earners completing: No additional credential", 
"Median earnings: Bachelor's degree earners with additional master's degree", 
"Percent of bachelor's degree earners completing: Additional masters degree", 
"Median earnings: Bachelor's degree earners with additional doctorate degree", 
"Percent of bachelor's degree earners completing: Additional doctorate degree",
"Formatted Major Code"]
#State University System of Florida Data
df = pd.read_excel('MFF_Singular_Sheet.xlsx', names=cols)

In [ ]:
#Crosswalk
uploaded = files.upload()
cols2 = ["SOC2018Code", "CIP2020Code"]
crosswalk = pd.read_excel('CIP_SOC.xlsx', names=cols2)

In [ ]:
#Occupation Data
uploaded = files.upload()
cols3 = ["AREA", "AREA_TITLE", "AREA_TYPE", "PRIM_STATE", "NAICS", "NAICS_TITLE",
         "I_GROUP", "OWN_CODE", "OCC_CODE", "OCC_TITLE", "O_GROUP",	
         "TOT_EMP", "EMP_PRSE", "JOBS_1000", "LOC_QUOTIENT", "PCT_TOTAL",
         "PCT_RPT", "H_MEAN", "A_MEAN"," MEAN_PRSE", "H_PCT10", "H_PCT25",	
         "H_MEDIAN", "H_PCT75", "H_PCT90", "A_PCT10", "A_PCT25", "A_MEDIAN", 
         "A_PCT75", "A_PCT90", "ANNUAL", "HOURLY"]
jobs = pd.read_excel('Florida Jobs.xlsx', names=cols3)

In [ ]:
#Major Group
uploaded = files.upload()
cols4 = ["Major group code", "Major group"]
group_codes = pd.read_excel('Major_Group_Codes.xlsx', names=cols4)
df1 = df
df1['Major code'] = df1['Major code'].str.slice(0, 3) #this condenses the codes so that they can be combined
merged = df1.merge(group_codes, left_on='Major code', right_on='Major group code')
merged

In [ ]:
#Salary and Occupation Match
df = df.query("`Median earnings: full-time workers` != '**'")
merged = merged.query("`Median earnings: full-time workers` != '**'") #this filtering takes out results that are only partially filled out and cannot be used
def salary_search():
  #Introduction
  print("This search is a simplified version of the final product.")
  print("These results will produce the averaged earnings of full-time "\
        "workers from a selection of Florida universities.")
  print("In addition, occupations are matched to prior results through "\
        "the Standard Occupational Classification (SOC) Code system and "\
         "their corresponding salaries are provided.")
  #Specific Major or Major Group
  print("Would you like to look at results of specified majors or " \
        "their groups?")
  input_choice = str(input("Enter 'Major' or 'Major Groups': "))

  if input_choice == 'Major':
    #University Selection
    print("Below is the current selection of universities. Please copy and paste " \
          "the one that you identify with the most")
    print(df["Institution"].unique())
    input_university = str(input('Enter your university: '))
    selected_university = df.query("Institution == @input_university")

    #Major Selection
    print("Below is the current selection of majors. Please copy and paste the " \
          "one that you identify with the most")
    print(selected_university["Major"].unique())
    input_major = str(input('Enter your major: '))
    selected_major = selected_university.query("Major == @input_major")
    
    #Degree Selection
    print("Below is the current selection of degrees. Please copy and paste the " \
          "one that you identify with the most")
    print(selected_major["Degree Level"].unique())
    input_degree = str(input('Enter your degree: '))
    selected_degree = selected_major.query("`Degree Level` == @input_degree") 

    #Cohort Year Selection
    print("Below is the current selection of cohort years. Please copy and " \
          "paste the one that you identify with the most")
    print(selected_degree["Cohort Year"].unique())
    input_cohort_year = str(input('Enter your cohort year: '))
    selected_cohort_year = selected_degree.query("`Cohort Year` == @input_cohort_year")
    
    #Major to Salary
    low_salary = selected_cohort_year['25th percentile earnings: full-time workers'].values[0] #The ".values[0]" gets rid of unwanted indicies that would ruin the format of the results
    median_salary = selected_cohort_year['Median earnings: full-time workers'].values[0] 
    high_salary = selected_cohort_year['75th percentile earnings: full-time workers'].values[0]
    print("The 25th percentile for this selection makes $" + str(low_salary))
    print("The median for this selection makes $" + str(median_salary))
    print("The 75th percentile for this selection makes $" + str(high_salary))

    #Possible Jobs That Match Major
    print("These are corresponding occupations and their median salary")
    selection = selected_cohort_year['Formatted Major Code'].values[0]
    SOC_array = crosswalk.query("CIP2020Code == @selection").values[0:,0] #"[0:,0]" obtains all the SOC codes, not just the first match
    SOC_length = len(SOC_array)
    salary_list = []
    title_list = []
    length_counter = 0
    while SOC_length > length_counter:
      try:
        SOC_slice = SOC_array[length_counter]
        job_selection = jobs.query("OCC_CODE == @SOC_slice")
        job_salary = job_selection['A_MEDIAN'].values[0]
        salary_list.append(job_salary)
        job_title = job_selection['OCC_TITLE'].values[0]
        title_list.append(job_title)
      except IndexError: #There are codes that don't necesarily match a job in this particular dataset, so this helps it skip over that without error
        pass
      length_counter += 1
    salary_list, title_list = zip(*sorted(zip(salary_list, title_list))) #Puts the arrays into accending order with salary_list being used as a reference for title_list, so that the integrity of the info is preserved
    list_len =  len(salary_list)
    while list_len > 0: #Reads results in a descending manner
      print("The occupation is " + str(title_list[list_len-1]) + " and the median amount is "
          + "$" + str(salary_list[list_len-1]))
      list_len -= 1
  if input_choice == 'Major Groups':
        #University Selection
    print("Below is the current selection of universities. Please copy and paste " \
          "the one that you identify with the most")
    print(merged["Institution"].unique())
    input_university = str(input('Enter your university: '))
    selected_university = merged.query("Institution == @input_university")

    #Major Group Selection
    print("Below is the current selection of major groups. Please copy and " \
          "paste the one that you identify with the most")
    print(selected_university["Major group"].unique())
    input_major = str(input('Enter your major group: '))
    selected_major = selected_university.query("`Major group` == @input_major")
    
    #Degree Selection
    print("Below is the current selection of degrees. Please copy and paste the " \
          "one that you identify with the most")
    print(selected_major["Degree Level"].unique())
    input_degree = str(input('Enter your degree: '))
    selected_degree = selected_major.query("`Degree Level` == @input_degree") 

    #Cohort Year Selection
    print("Below is the current selection of cohort years. Please copy and " \
          "paste the one that you identify with the most")
    print(selected_degree["Cohort Year"].unique())
    input_cohort_year = str(input('Enter your cohort year: '))
    selected_cohort_year = selected_degree.query("`Cohort Year` == @input_cohort_year")

    #Calculating Weighted Median
    grads = selected_cohort_year["Graduates (with full-time wages)"].values[0:]
    med_earn = selected_cohort_year["Median earnings: full-time workers"].values[0:]
    weights = sum(grads * med_earn)
    div = sum(grads)
    weighted_med = round(weights / div)
    print("The weighted median for this selection makes $" + str(weighted_med))
  
  #Possible Jobs That Match Major
    print("These are corresponding occupations and their median salary")
    selection = selected_cohort_year['Formatted Major Code'].values[0:]
    selection_length = 0
    salary_list = []
    title_list = []
    while len(selection) > selection_length:
      SOC_array = crosswalk.query("CIP2020Code == @selection[0+@selection_length]").values[0:,0]
      SOC_length = len(SOC_array)
      length_counter = 0
      while SOC_length > length_counter:
        try:
          SOC_slice = SOC_array[length_counter]
          job_selection = jobs.query("OCC_CODE == @SOC_slice")
          job_title = job_selection['OCC_TITLE'].values[0]
          if job_title in title_list: #This gets rid of duplicates
            pass
          else:
            title_list.append(job_title)
            job_salary = job_selection['A_MEDIAN'].values[0]
            salary_list.append(job_salary)
        except IndexError:
          pass
        length_counter += 1
      selection_length += 1
    salary_list, title_list = zip(*sorted(zip(salary_list, title_list)))
    list_len =  len(salary_list)
    while list_len > 0:
      print("The occupation is " + str(title_list[list_len-1]) + " and the median amount is "
          + "$" + str(salary_list[list_len-1]))
      list_len -= 1

salary_search()

In [ ]:
#Compare Search
def compare_search():
  #Introduction
  print("This search is a simplified version of the final product.")
  print("These results will compare a selected major and their averaged salary"\
        " and how they fare based on the Florida university attended.")
  filtered_data = df.query("`Median earnings: full-time workers` != '**'") 
  #Major Selection
  print("Below is the current selection of majors. Please copy and paste the " \
        "one that you identify with the most")
  print(filtered_data["Major"].unique())
  input_major = str(input('Enter your major: '))
  selected_major = filtered_data.query("Major == @input_major")
  
  #Degree Selection
  print("Below is the current selection of degrees. Please copy and paste the " \
        "one that you identify with the most")
  print(selected_major["Degree Level"].unique())
  input_degree = str(input('Enter your degree: '))
  selected_degree = selected_major.query("`Degree Level` == @input_degree") 
  #Taking System Out of Results For Plotting
  no_system = selected_degree.query("Institution != 'System'")
  #25th percentile
  plt.figure(figsize=(20,7))
  plot = sns.barplot(x = 'Institution', 
                     y = '25th percentile earnings: full-time workers', 
                     hue='Cohort Year',  data = no_system)
  for i in plot.containers:
    plot.bar_label(i,)
  #selected_system25_1 = selected_degree.query("Institution == 'System' & `Cohort Year` == '1 year after graduation'")
  #plt.axhline(y=selected_system25_1['25th percentile earnings: full-time workers'].values[0], ls='--', c = '#1f77b4')
  #selected_system25_5 = selected_degree.query("Institution == 'System' & `Cohort Year` == '5 years after graduation'")
  #plt.axhline(y=selected_system25_5['25th percentile earnings: full-time workers'].values[0], ls='--', c = '#ff7f0e')
  #selected_system25_10 = selected_degree.query("Institution == 'System' & `Cohort Year` == '10 years after graduation'")
  #plt.axhline(y=selected_system25_10['25th percentile earnings: full-time workers'].values[0], ls='--', c = '#2ca02c')
  plt.title("25th Percentile for " + str(input_degree) + " in " + str(input_major))
  plt.show()
  #Median percentile
  plt.figure(figsize=(20,7))
  plot = sns.barplot(x = 'Institution', 
                     y = 'Median earnings: full-time workers', 
                     hue='Cohort Year',  data = no_system)
  for i in plot.containers:
    plot.bar_label(i,)
  #selected_system_med_1 = selected_degree.query("Institution == 'System' & `Cohort Year` == '1 year after graduation'")
  #plt.axhline(y=selected_system_med_1['Median earnings: full-time workers'].values[0], ls='--', c = '#1f77b4')
  #selected_system_med_5 = selected_degree.query("Institution == 'System' & `Cohort Year` == '5 years after graduation'")
  #plt.axhline(y=selected_system_med_5['Median earnings: full-time workers'].values[0], ls='--', c = '#ff7f0e')
  #selected_system_med_10 = selected_degree.query("Institution == 'System' & `Cohort Year` == '10 years after graduation'")
  #plt.axhline(y=selected_system_med_10['Median earnings: full-time workers'].values[0], ls='--', c = '#2ca02c')
  plt.title("Median Percentile for " + str(input_degree) + " in " + str(input_major))
  plt.show()
  #75th percentile
  plt.figure(figsize=(20,7))
  plot = sns.barplot(x = 'Institution', 
                     y = '75th percentile earnings: full-time workers', 
                     hue='Cohort Year',  data = no_system)
  for i in plot.containers:
    plot.bar_label(i,)
  selected_system75_1 = selected_degree.query("Institution == 'System' & `Cohort Year` == '1 year after graduation'")
  #plt.axhline(y=selected_system75_1['75th percentile earnings: full-time workers'].values[0], ls='--', c = '#1f77b4')
  #selected_system75_5 = selected_degree.query("Institution == 'System' & `Cohort Year` == '5 years after graduation'")
  #plt.axhline(y=selected_system75_5['75th percentile earnings: full-time workers'].values[0], ls='--', c = '#ff7f0e')
  #selected_system75_10 = selected_degree.query("Institution == 'System' & `Cohort Year` == '10 years after graduation'")
  #plt.axhline(y=selected_system75_10['75th percentile earnings: full-time workers'].values[0], ls='--', c = '#2ca02c')
  plt.title("75th Percentile for " + str(input_degree) + " in " + str(input_major))
  plt.show()

compare_search()

In [ ]:
#What Percent Continues Education
def continuing_education():
  #Introduction
  print("This search is a simplified version of the final product.")
  print("These results will produce the rate of those with a bachelors degree "\
        "who continue their education during cohort year 1.")
  filtered_data = df.query("`Degree Level` == 'Bachelors' & "\
                           "`Cohort Year` == '1 year after graduation' & "\
                           "`Percent of graduates continuing education` != '**'") 
  #Major Selection
  print("Below is the current selection of majors. Please copy and paste the " \
        "one that you identify with the most")
  print(filtered_data["Major"].unique())
  input_major = str(input('Enter your major: '))
  selected_major = filtered_data.query("Major == @input_major")
  #Taking System Out of Results For Plotting
  no_system = selected_major.query("Institution != 'System'")
  #Sorting
  sort_plot = no_system.sort_values('Percent of graduates continuing education', ascending=True)
  #Plot
  plt.figure(figsize=(20,7))
  plot = sns.barplot(x = 'Institution', 
                     y = 'Percent of graduates continuing education', 
                     data = sort_plot)
  for i in plot.containers:
    plot.bar_label(i,)
  selected_system = selected_major.query("Institution == 'System'")
  plt.axhline(y=selected_system['Percent of graduates continuing education'].values[0], ls='--')
  plt.title("Percent of Graduates Continuing Education Beyond a Bachelors Degree in " + str(input_major))
  plt.show()

continuing_education()

In [ ]:
#If you get errors above install these and restart the run time
!pip install matplotlib --upgrade
!pip install seaborn --upgrade